```{contents}
```
## Async Execution

### What Is Async Execution

**Async execution** allows a runnable to execute **non-blocking operations concurrently** using Python’s `async/await`.
In LangChain, most runnables support async methods such as **`.ainvoke()`**, **`.abatch()`**, and **`.astream()`**.

Async execution is critical for:

* High concurrency
* I/O-bound workloads (LLMs, DBs, APIs)
* Web frameworks (FastAPI, async Django views)

Supported natively by LangChain.

```
Multiple Requests
   ↓
Async Event Loop
   ↓
Concurrent Runnable Execution
```

---

### Why Async Execution Is Needed

Without async:

* Each request blocks the process
* Poor throughput
* Wasted CPU while waiting on I/O

With async:

* One worker handles many requests
* Better latency under load
* Scales efficiently in APIs

---

### Sync vs Async (Conceptual)

| Aspect      | Sync       | Async             |
| ----------- | ---------- | ----------------- |
| Blocking    | Yes        | No                |
| Concurrency | Limited    | High              |
| Best for    | Scripts    | APIs / servers    |
| Pattern     | `invoke()` | `await ainvoke()` |

---

### Basic Async Execution (RunnableLambda)



In [2]:
from langchain_core.runnables import RunnableLambda
import asyncio

async_runnable = RunnableLambda(lambda x: x.upper())

async def run():
    result = await async_runnable.ainvoke("async execution")
    print(result)

# In Jupyter notebooks, use await directly instead of asyncio.run()
await run()


ASYNC EXECUTION




**Output**

```
ASYNC EXECUTION
```

> **Note:** In Jupyter notebooks, use `await run()` directly instead of `asyncio.run(run())` because notebooks already have a running event loop. Use `asyncio.run()` only in standalone Python scripts.

---

### Async Execution with RunnableSequence



In [3]:
from langchain_core.runnables import RunnableLambda

chain = (
    RunnableLambda(lambda x: x.strip())
    | RunnableLambda(lambda x: x.lower())
)

async def run():
    result = await chain.ainvoke("  ASYNC PIPELINE ")
    print(result)

# In notebooks: await run()
# In scripts: asyncio.run(run())
await run()


async pipeline




**Execution**

* Steps still run **in order**
* Entire sequence is **non-blocking**

---

### Async Execution with LLMs



In [4]:
from langchain_openai import ChatOpenAI
import asyncio

llm = ChatOpenAI()

async def run():
    response = await llm.ainvoke("Explain async execution briefly")
    print(response.content)

# In notebooks: await run()
# In scripts: asyncio.run(run())
await run()


Async execution is a programming concept that allows tasks to be executed concurrently, without being blocked by other tasks. This means that instead of waiting for one task to complete before moving on to the next, multiple tasks can be processed at the same time. This can help improve the performance and efficiency of a program by reducing wait times and increasing productivity.




This is mandatory when:

* Using FastAPI
* Handling many parallel users

---

### Async Batch Execution (`abatch`)



In [5]:
async def run():
    results = await llm.abatch([
        "Explain RAG",
        "Explain agents",
        "Explain memory"
    ])
    for r in results:
        print(r.content)

await run()


RAG stands for Red, Amber, Green and is a color-coded system used to assess and communicate the status of various items or situations. 

- Red typically indicates that there is a critical issue or problem that needs urgent attention and resolution.
- Amber signals that there may be some concerns or potential risks that need to be monitored and managed.
- Green indicates that everything is running smoothly and there are no significant issues to report.

RAG is commonly used in project management, risk assessment, and performance tracking to quickly convey the status of different areas. It helps stakeholders easily understand the current situation and make informed decisions based on the color-coded indicators.
Agents are individuals or entities that act on behalf of someone else or represent a company or organization. They are typically authorized to make decisions, negotiate deals, or perform other tasks on behalf of their clients. 

In business or real estate, agents may represent eit



Benefits:

* Concurrent LLM calls
* Lower total latency
* Higher throughput

---

### Async Streaming (`astream`)

```python
async def run():
    async for chunk in llm.astream("Explain async streaming"):
        print(chunk.content, end="")

await run()
```

Used for:

* Real-time UI updates
* SSE / WebSocket responses

---

### Async with RunnableParallel



In [6]:
from langchain_core.runnables import RunnableParallel

parallel = RunnableParallel(
    upper=RunnableLambda(lambda x: x.upper()),
    length=RunnableLambda(lambda x: len(x))
)

async def run():
    result = await parallel.ainvoke("async parallel")
    print(result)

await run()


{'upper': 'ASYNC PARALLEL', 'length': 14}




**Output**

```python
{"upper": "ASYNC PARALLEL", "length": 14}
```

Branches execute concurrently.

---

### Async Execution in FastAPI (Real-World)

```python
from fastapi import FastAPI

app = FastAPI()

@app.get("/chat")
async def chat(q: str):
    response = await llm.ainvoke(q)
    return {"answer": response.content}
```

Why async is mandatory here:

* FastAPI is async-first
* Blocking calls degrade performance

---

### What Supports Async in LangChain

| Component             | Async         |
| --------------------- | ------------- |
| LLMs                  | ✅             |
| RunnableSequence      | ✅             |
| RunnableParallel      | ✅             |
| Batch (`abatch`)      | ✅             |
| Streaming (`astream`) | ✅             |
| Prompt templates      | ❌ (sync only) |

---

### Common Mistakes

* Calling `.invoke()` inside async functions
* Forgetting `await`
* Mixing blocking I/O with async
* Using `asyncio.run()` inside Jupyter notebooks (use `await` directly)
* Using `await` in standalone scripts without wrapping in `asyncio.run()`

---

### Notebook vs Script Execution

**In Jupyter Notebooks:**
```python
async def my_function():
    return await some_async_operation()

# Use await directly
await my_function()
```

**In Python Scripts:**
```python
import asyncio

async def my_function():
    return await some_async_operation()

# Wrap in asyncio.run()
asyncio.run(my_function())
```

---

### Mental Model

Async execution turns:

```
One request → wait → next request
```

into:

```
Many requests → overlap waits → efficient execution
```

---

### When to Use Async Execution

Use it when:

* Building APIs
* Handling multiple users
* Calling LLMs, DBs, or APIs
* Streaming responses

Avoid it when:

* Writing simple scripts
* CPU-bound workloads

---

### Key Takeaways

* Async execution enables **non-blocking concurrency**
* Use `.ainvoke()`, `.abatch()`, `.astream()`
* Essential for scalable LLM applications
* Works seamlessly with LCEL pipelines
* **In notebooks:** use `await` directly; **In scripts:** use `asyncio.run()`
